In [1]:
DATA_PATH="/Users/telnetning/Downloads/data/zhwiki/"

In [2]:
# 暂时使用整个 AASIMP（AA转换成简体后的文件目录） 目录下的 100 个文件: wiki_00 ~ wiki_99
import os
# for file in os.listdir(DATA_PATH + "AASIMP") 

In [10]:
import modin.pandas as pd

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-11_00-31-47_26801/logs.
Waiting for redis server at 127.0.0.1:11504 to respond...
Waiting for redis server at 127.0.0.1:48807 to respond...
Starting Redis shard with 10.0 GB max memory.
Starting the Plasma object store with 5.0 GB memory using /tmp.


In [11]:
all_files = [file for file in os.listdir(DATA_PATH + "AASIMP")]

In [12]:
print(all_files)

['wiki_73', 'wiki_87', 'wiki_80', 'wiki_74', 'wiki_89', 'wiki_42', 'wiki_45', 'wiki_11', 'wiki_16', 'wiki_29', 'wiki_20', 'wiki_27', 'wiki_18', 'wiki_44', 'wiki_88', 'wiki_43', 'wiki_75', 'wiki_81', 'wiki_86', 'wiki_72', 'wiki_26', 'wiki_19', 'wiki_21', 'wiki_17', 'wiki_28', 'wiki_10', 'wiki_32', 'wiki_35', 'wiki_03', 'wiki_04', 'wiki_50', 'wiki_57', 'wiki_68', 'wiki_61', 'wiki_95', 'wiki_92', 'wiki_66', 'wiki_59', 'wiki_05', 'wiki_02', 'wiki_34', 'wiki_33', 'wiki_67', 'wiki_93', 'wiki_58', 'wiki_94', 'wiki_60', 'wiki_56', 'wiki_69', 'wiki_51', 'wiki_15', 'wiki_12', 'wiki_24', 'wiki_23', 'wiki_48', 'wiki_83', 'wiki_77', 'wiki_70', 'wiki_84', 'wiki_79', 'wiki_46', 'wiki_41', 'wiki_22', 'wiki_25', 'wiki_13', 'wiki_14', 'wiki_40', 'wiki_78', 'wiki_47', 'wiki_85', 'wiki_71', 'wiki_49', 'wiki_76', 'wiki_82', 'wiki_54', 'wiki_53', 'wiki_98', 'wiki_91', 'wiki_65', 'wiki_62', 'wiki_96', 'wiki_09', 'wiki_36', 'wiki_31', 'wiki_38', 'wiki_07', 'wiki_00', 'wiki_97', 'wiki_63', 'wiki_64', 'wiki_90'

In [19]:
# 对文章进行处理的函数
import re
def token(string):
    return ' '.join(re.findall('[\w|\d]+', string))

In [20]:
data = ''
for file in all_files:
    with open(DATA_PATH + "AASIMP/" + file) as f:
        data += token(f.read())

In [21]:
print(type(data))

<class 'str'>


In [22]:
print(len(data))

37162299


### 分词

In [23]:
import jieba

In [24]:
def cut(string): return list(jieba.cut(string))

In [26]:
ALL_TOKNES = cut(data)

Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/8b/_681s2t12zzfg84s_yp6lfm00000gn/T/jieba.cache
Loading model from cache /var/folders/8b/_681s2t12zzfg84s_yp6lfm00000gn/T/jieba.cache
Loading model cost 1.174 seconds.
Loading model cost 1.174 seconds.
Prefix dict has been built succesfully.
Prefix dict has been built succesfully.


In [28]:
print(len(ALL_TOKNES))

20965362


In [29]:
valid_token = [t for t in ALL_TOKNES if t.strip() and t != 'n'] # 去除空白字符和换行符

In [30]:
len(valid_token)

17220910

In [31]:
valid_token[:20]

['doc',
 'id',
 '47081',
 'url',
 'https',
 'zh',
 'wikipedia',
 'org',
 'wiki',
 'curid',
 '47081',
 'title',
 '公有',
 '领域',
 '公有',
 '领域',
 '公有',
 '领域',
 '是',
 '人类']

In [32]:
from collections import Counter
words_count = Counter(valid_token)

In [33]:
words_count.most_common(10)

[('的', 984867),
 ('在', 255319),
 ('年', 207999),
 ('是', 182701),
 ('和', 157811),
 ('了', 125518),
 ('为', 115956),
 ('与', 91380),
 ('有', 87706),
 ('月', 86406)]

In [36]:
# most_common 返回前 x 个频次最高的，返回类型 [(word1, frequency1), (word2, frequency2)...]
all_frequences = [f for w, f in words_count.most_common()]

In [37]:
frequences_sum = sum(all_frequences)
# 某个单词出现的概率 = 该单词出现的次数 / 所有单词一共出现的次数
def get_prob(word):
    return words_count[word] / frequences_sum if word in words_count else 1 / frequences_sum

In [38]:
get_prob('我们')

0.00030753310945821097

In [39]:
get_prob('存在')

0.0004489890487784908

In [40]:
 get_prob('数学')

0.00019424060633265025

In [41]:
get_prob('火锅')

4.297101604967449e-06

In [42]:
## 以上可以基本看出，词频和日常生活的词频可能不太一样

### 1-Gram

In [44]:
from functools import reduce
def product(numbers):
    return reduce(lambda n1, n2: n1 * n2, numbers)

def language_model_one_gram(sentence):
    return product(get_prob(word) for word in cut(sentence))

In [45]:
language_model_one_gram('今天我不想上班')

3.896419478569954e-18

In [46]:
language_model_one_gram('明天天气不错')

3.7987803611338926e-15

In [47]:
language_model_one_gram('这是一个数学问题')

6.194485194474578e-14

In [48]:
# 条件概率，A事件出现的情况下B事件出现的概率 = (AB事件出现的概率) / A事件出现的概率
all_2_grams_words = [''.join(valid_token[i:i+2]) for i in range(len(valid_token[:-2]))] # AB连着出现

_2_gram_sum = len(all_2_grams_words)
_2_gram_counter = Counter(all_2_grams_words)

def get_combination_prob(w1, w2):
    # 联合概率
    if w1 + w2 in all_2_grams_words:
         return _2_gram_counter[w1+w2] / _2_gram_sum
    else:
        return 1 / _2_gram_sum

In [58]:
all_2_grams_words[:20]

['docid',
 'id47081',
 '47081url',
 'urlhttps',
 'httpszh',
 'zhwikipedia',
 'wikipediaorg',
 'orgwiki',
 'wikicurid',
 'curid47081',
 '47081title',
 'title公有',
 '公有领域',
 '领域公有',
 '公有领域',
 '领域公有',
 '公有领域',
 '领域是',
 '是人类',
 '人类的']

In [59]:
get_combination_prob('公有','领域')

4.529377893430474e-06

In [55]:
get_combination_prob('回','深圳')

5.806894735167275e-08

In [56]:
get_combination_prob('世界','大战')

5.806894735167275e-08

In [62]:
def get_prob_2_gram(w1, w2):
    return get_combination_prob(w1, w2) / get_prob(w1)

In [63]:
def language_model_of_2_gram(sentence):
    sentence_probability = 1
    
    words = cut(sentence)
    
    for i, word in enumerate(words):
        if i == 0: 
            prob = get_prob(word) 
        else:
            previous = words[i-1]
            prob = get_prob_2_gram(previous, word)
        sentence_probability *= prob # 概率相乘
    
    return sentence_probability

In [64]:
language_model_of_2_gram('明天要加班')

1.1125386358018594e-11

In [65]:
language_model_of_2_gram('明天不加班')

1.8155058338495557e-12

In [66]:
language_model_of_2_gram('小猫吃鱼')

4.23517493404459e-10

In [67]:
language_model_of_2_gram('鱼吃老虎')

4.7057499267162104e-11

In [68]:
language_model_of_2_gram('鱼吃小猫')

4.7057499267162104e-11

In [71]:
language_model_of_2_gram('数学是一门学科')

1.4306767116841882e-10

In [72]:
language_model_of_2_gram('宇宙很大')

5.806894735167275e-08

In [73]:
### 数据量太小，合理性有限，并且数据太靠近科学，而远离生活，场景有限

### 其它

+  数据量不仅要大，而且要来源广泛，才可能更准确
+  如果可以区分一个问题的场景，采用对应场景的数据集，可能得到的结果会更准确
+  工程处理上，大量数据，可以采用线下分词，尤其对于 1-Gram，可以线下分词存到数据库，线上直接访问数据库查频次
+  modin 比 pandas 更快，简繁转换，出了 linux 自带 opencc，还有 hanziconv 